<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-20 10:21:46
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.16 L
Current:  1.45 C
-------------------
Today PnL: 18.03 K (0.12%)
Current PnL: -21.10 L (-13.83%)
CY Booked + Current PnL: -7.14 L (-4.68%)
-------------------
Total profit:  1.79 L
Total loss:  -22.89 L
-------------------
Total Booked + Current PnL: 19.84 L (15.79%)
Total Booked PnL: 40.94 L (32.59%)
Curr Year Booked PnL: 13.96 L (9.65%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 86.01 L (59.45%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.12%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.46,-6.77,7.28,0.02,11848.0,-11815.0,162744.0,148.33,78.0,M-SC,3.09,234.0,-1.00,1.13,41.56,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.14,-14.21,16.67,0.09,14413.0,-14317.0,86460.0,100.64,49.0,M-SC,4.03,253.0,-0.99,0.60,13.59,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,0.23,9.91,8.90,19.69,18757.0,19008.0,210750.0,-0.80,57.0,X-MC,3.64,78.0,1.01,1.47,17.38,XY25,NTT,AC
35,ICICIGI,2252.93,-0.08,8.84,10.43,20.19,19789.0,15409.0,189729.0,-14.60,58.0,X-MC,7.69,68.0,0.78,1.32,25.29,X40,ATH,INSURANCE
53,NESTLEIND,1377.00,0.31,13.11,7.33,21.40,22006.0,34796.0,300222.0,4.93,64.0,X-LC,8.07,12.0,1.58,2.09,20.62,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,2.76,5.80,17.94,24.79,47806.0,14619.0,266475.0,0.98,63.0,X-SC,2.28,86.0,0.31,1.86,30.29,X40N,NTT,MISC
0,5PAISA,593.0,2.67,-34.78,73.11,12.90,104182.0,-75995.0,142501.0,110.34,63.0,H-SC,15.68,161.0,-0.73,0.99,18.49,OX40N,NTT,FINANCE
37,IEX,219.0,2.03,-5.06,56.54,48.63,107338.0,-10110.0,189844.0,-36.95,50.0,H-SC,15.33,136.0,-0.09,1.32,5.91,XR,NTT,MISC
57,RAJESHEXPO,518.0,1.75,-62.88,169.57,0.07,86348.0,-86255.0,50922.0,1766.16,65.0,L-SC,2.23,268.0,-1.00,0.35,26.65,OX40N,NTT,JEWELLERY
33,HINDUNILVR,2922.0,1.56,-5.02,19.68,13.67,48051.0,-12903.0,244160.0,-12.11,46.0,X-LC,4.81,14.0,-0.27,1.70,15.26,XY25,NTT,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.00,-2.69,-43.68,209.23,74.15,192785.0,-71465.0,92140.0,-265.94,30.0,H-SC,13.33,149.0,-0.37,0.64,18.11,XR,NTT,CHEMICALS
56,QUESS,424.00,-2.31,-26.61,94.63,42.84,45147.0,-17297.0,47709.0,-52.07,41.0,X-SC,37.15,83.0,-0.38,0.33,3.37,XY24,NTT,MISC
14,CAMPUS,393.00,-2.03,-12.53,52.38,33.28,76865.0,-21030.0,146745.0,-29.55,31.0,M-SC,3.04,222.0,-0.27,1.02,13.74,XY24,NTT,FOOTWEAR
72,TANLA,1943.92,-1.64,-32.19,223.99,119.69,411246.0,-87161.0,183600.0,-31.42,37.0,H-SC,12.87,156.0,-0.21,1.28,45.04,AR,ATH,IT
80,VAIBHAVGBL,521.00,-1.59,-23.66,110.59,60.77,154311.0,-43241.0,139534.0,60.88,46.0,H-SC,11.07,159.0,-0.28,0.97,25.66,XR,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.17,-1.92,111.99,107.91,162165.0,-2841.0,144803.0,-20.15,42.0,M-SC,10.11,216.0,-0.02,1.01,1.02,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.46,-6.77,7.28,0.02,11848.0,-11815.0,162744.0,148.33,78.0,M-SC,3.09,234.0,-1.00,1.13,41.56,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.14,-14.21,16.67,0.09,14413.0,-14317.0,86460.0,100.64,49.0,M-SC,4.03,253.0,-0.99,0.60,13.59,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.52,-19.52,40.99,13.47,88963.0,-52632.0,217035.0,-66.95,37.0,H-SC,1.75,158.0,-0.59,1.51,10.36,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,0.09,-22.86,71.15,32.03,72358.0,-30137.0,101698.0,-16.54,49.0,H-SC,6.41,148.0,-0.42,0.71,13.91,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.13,1.21,70.45,72.51,107625.0,1823.0,152768.0,-13.34,56.0,M-LC,5.08,99.0,0.02,1.06,7.30,XR,NTT,IT
38,INDIAMART,4810.62,-0.17,3.52,99.69,106.72,127286.0,4346.0,127682.0,-48.72,44.0,H-SC,4.60,139.0,0.03,0.89,27.73,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.17,3.52,99.69,106.72,127286.0,4346.0,127682.0,-48.72,44.0,H-SC,4.60,139.0,0.03,0.89,27.73,AR,ATH,MISC
85,WIPRO,420.00,0.13,1.21,70.45,72.51,107625.0,1823.0,152768.0,-13.34,56.0,M-LC,5.08,99.0,0.02,1.06,7.30,XR,NTT,IT
25,FINCABLES,1641.55,-0.17,-1.92,111.99,107.91,162165.0,-2841.0,144803.0,-20.15,42.0,M-SC,10.11,216.0,-0.02,1.01,1.02,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,0.63,-1.29,37.40,35.63,77372.0,-2696.0,206878.0,-15.41,37.0,H-MC,5.03,119.0,-0.03,1.44,15.17,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,-0.85,-4.48,90.45,81.92,99183.0,-5143.0,109655.0,-44.03,33.0,M-SC,6.97,236.0,-0.05,0.76,29.81,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,0.21,-14.31,65.93,42.18,156154.0,-39562.0,236848.0,-23.08,22.0,X-LC,0.03,3.0,-0.25,1.65,0.21,XY24,NTT,AUTO
11,BATAINDIA,2096.0,0.29,-34.54,106.54,35.20,89737.0,-44442.0,84228.0,10.47,25.0,X-SC,11.46,92.0,-0.50,0.59,0.29,X40,NTT,FOOTWEAR
60,RELAXO,1176.0,-0.30,-46.60,185.19,52.31,143563.0,-67638.0,77522.0,-43.18,31.0,X-SC,3.29,91.0,-0.47,0.54,3.77,X40N,NTT,FOOTWEAR
9,BAJAJHFL,181.5,0.13,-13.14,70.90,48.45,123485.0,-26338.0,174168.0,-19.44,31.0,X-MC,7.10,64.0,-0.21,1.21,0.44,X40N,ATH,FINANCE
46,JSWINFRA,342.0,0.23,-6.35,23.04,15.23,43227.0,-12731.0,187616.0,-19.93,32.0,X-MC,6.15,66.0,-0.29,1.31,23.85,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,0.21,-14.31,65.93,42.18,156154.0,-39562.0,236848.0,-23.08,22.0,X-LC,0.03,3.0,-0.25,1.65,0.21,XY24,NTT,AUTO
43,ITC,452.00,0.59,-0.81,11.34,10.44,22511.0,-1628.0,198510.0,-40.74,46.0,X-LC,0.94,5.0,-0.07,1.38,4.70,X40,NTT,FMCG
34,HONAUT,58357.33,0.50,-18.00,66.88,36.84,70164.0,-23028.0,104910.0,-26.21,32.0,X-SC,1.42,90.0,-0.33,0.73,7.40,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,0.17,1.96,28.46,30.97,51283.0,3457.0,180195.0,-22.06,52.0,X-MC,1.43,75.0,0.07,1.26,14.76,X40,ATH,INSURANCE
66,SIEMENS,4671.50,0.11,-13.62,45.31,25.51,72834.0,-25350.0,160745.0,-1.39,59.0,H-LC,1.82,49.0,-0.35,1.12,18.46,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,PAGEIND,51605.08,-0.09,-5.28,33.09,26.07,51323.0,-8640.0,155100.0,-30.18,32.0,X-MC,8.44,55.0,-0.17,1.08,0.00,X40,ATH,APPARELS
76,TMPV,600.00,0.21,-14.31,65.93,42.18,156154.0,-39562.0,236848.0,-23.08,22.0,X-LC,0.03,3.0,-0.25,1.65,0.21,XY24,NTT,AUTO
11,BATAINDIA,2096.00,0.29,-34.54,106.54,35.20,89737.0,-44442.0,84228.0,10.47,25.0,X-SC,11.46,92.0,-0.50,0.59,0.29,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,0.13,-13.14,70.90,48.45,123485.0,-26338.0,174168.0,-19.44,31.0,X-MC,7.10,64.0,-0.21,1.21,0.44,X40N,ATH,FINANCE
55,PGHH,17907.65,0.29,-3.32,38.36,33.76,74474.0,-6675.0,194145.0,-31.84,35.0,X-MC,2.86,57.0,-0.09,1.35,1.70,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-2.31,-26.61,94.63,42.84,45147.0,-17297.0,47709.0,-52.07,41.0,X-SC,37.15,83.0,-0.38,0.33,3.37,XY24,NTT,MISC
60,RELAXO,1176.00,-0.30,-46.60,185.19,52.31,143563.0,-67638.0,77522.0,-43.18,31.0,X-SC,3.29,91.0,-0.47,0.54,3.77,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.29,-34.54,106.54,35.20,89737.0,-44442.0,84228.0,10.47,25.0,X-SC,11.46,92.0,-0.50,0.59,0.29,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,0.50,-18.00,66.88,36.84,70164.0,-23028.0,104910.0,-26.21,32.0,X-SC,1.42,90.0,-0.33,0.73,7.40,X40N,ATH,ELECTRICAL
52,MEDANTA,1486.00,-0.33,-0.22,24.35,24.07,26770.0,-247.0,109940.0,-8.38,37.0,X-SC,4.66,89.0,-0.01,0.77,19.35,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.30,-14.22,39.06,19.28,115914.0,-49198.0,296758.0,-25.24,64.0,X-LC,7.94,1.0,-0.42,2.07,9.71,X40,ATH,IT
41,INFY,2275.00,-0.03,4.88,47.67,54.87,156428.0,15260.0,328148.0,-18.16,59.0,X-LC,3.42,2.0,0.10,2.29,13.56,X40,BTT,IT
76,TMPV,600.00,0.21,-14.31,65.93,42.18,156154.0,-39562.0,236848.0,-23.08,22.0,X-LC,0.03,3.0,-0.25,1.65,0.21,XY24,NTT,AUTO
82,VBL,671.64,-0.43,-8.57,48.40,35.67,139759.0,-27083.0,288759.0,-19.52,44.0,X-LC,6.01,4.0,-0.19,2.01,4.09,X40N,ATH,FMCG
43,ITC,452.00,0.59,-0.81,11.34,10.44,22511.0,-1628.0,198510.0,-40.74,46.0,X-LC,0.94,5.0,-0.07,1.38,4.70,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,1.18,-37.61,116.80,35.27,82750.0,-42701.0,70848.0,-51.71,34.0,L-SC,19.45,270.0,-0.52,0.49,73.40,XR,NTT,HOTELS
6,ASIANTILES,137.00,-0.37,-11.25,104.69,81.67,87166.0,-10549.0,83261.0,7488.89,57.0,L-SC,18.15,271.0,-0.12,0.58,61.24,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.04,-12.33,59.18,39.54,52481.0,-12478.0,88681.0,-31.43,70.0,M-SC,6.83,220.0,-0.24,0.62,25.05,AR,ATH,AUTO
51,MASFIN,398.61,-0.81,-4.62,27.96,22.05,26129.0,-4530.0,93450.0,-17.80,53.0,H-SC,7.16,164.0,-0.17,0.65,35.84,XR,ATH,FINANCE
80,VAIBHAVGBL,521.00,-1.59,-23.66,110.59,60.77,154311.0,-43241.0,139534.0,60.88,46.0,H-SC,11.07,159.0,-0.28,0.97,25.66,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.46,-6.77,7.28,0.02,11848.0,-11815.0,162744.0,148.33,78.0,M-SC,3.09,234.0,-1.00,1.13,41.56,OX40N,NTT,PAINTS
13,BSOFT,831.70,-0.46,-29.39,113.53,50.79,112760.0,-41331.0,99322.0,-9.55,59.0,H-SC,7.89,151.0,-0.37,0.69,15.64,XR,ATH,IT
5,ASIANPAINT,3460.26,-0.64,0.50,20.34,20.95,51465.0,1258.0,253026.0,-0.49,78.0,X-LC,11.50,31.0,0.02,1.76,36.31,X40,ATH,PAINTS
80,VAIBHAVGBL,521.00,-1.59,-23.66,110.59,60.77,154311.0,-43241.0,139534.0,60.88,46.0,H-SC,11.07,159.0,-0.28,0.97,25.66,XR,NTT,JEWELLERY
32,HCLTECH,1922.01,-0.23,7.64,15.87,24.73,41328.0,18492.0,260416.0,12.34,75.0,X-LC,4.51,7.0,0.45,1.81,24.04,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.91
1,25,44.93
2,50,75.79


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.78
MC    29.14
LC    25.06
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.06
X40      21.00
X40N     11.64
XR        9.52
XY25      9.15
AR        9.14
OX40N     7.74
X200      1.89
SR        1.02
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.35
X-MC    22.66
X-LC    20.02
M-SC    12.13
X-SC     7.88
H-MC     4.76
H-LC     3.01
L-SC     1.42
M-MC     1.39
M-LC     1.06
L-LC     0.97
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.83,-4.11,37.78
IT,13.70,-13.28,71.81
FINANCE,9.48,-13.88,62.00
MISC,7.52,-20.32,69.61
PAINTS,6.05,-6.24,22.02
ELECTRICAL,5.67,-10.95,50.29
INSURANCE,4.50,2.00,32.54
PHARMA,3.66,-0.69,33.64
AUTO,3.36,-19.35,70.53


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2988431.0,21
XR,1261585.0,13
AR,1261483.0,10
X40,989047.0,14
X40N,701234.0,9
OX40N,677796.0,10
XY25,327768.0,6
SR,275368.0,2
MH,66030.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3484325.0,25
M-SC,1315051.0,15
X-MC,1176495.0,16
X-LC,888118.0,11
X-SC,711477.0,8
H-MC,395505.0,3
L-SC,256264.0,3
H-LC,125557.0,2
M-LC,107625.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1189789.0      6
           AR         855181.0      5
           XR         781697.0      7
M-SC       XY24       771325.0      6
X-MC       X40        427982.0      7
X-LC       X40        387646.0      5
X-MC       XY24       342941.0      3
H-SC       OX40N      316260.0      4
X-SC       XY24       276525.0      3
H-SC       SR         275368.0      2
M-SC       OX40N      275188.0      5
X-SC       X40N       261533.0      3
X-MC       X40N       233884.0      4
X-LC       XY24       224598.0      2
H-MC       AR         212252.0      2
X-LC       X40N       205817.0      2
H-MC       XY24       183253.0      1
X-SC       X40        173419.0      2
X-MC       XY25       171688.0      2
L-SC       XR         169916.0      2
M-SC       XR         147322.0      2
           AR         121216.0      2
M-LC       XR         107625.0      1
L-SC       OX40N       86348.0      1
H-LC       AR          72834.0      1
X-LC       XY25        70057.0      2
H-SC       MH          66030.0      1
L-MC       XR          55025.0      1
H-LC       X200        52723.0      1
M-MC       XY25        46499.0      1
L-LC       XY25        39524.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
